This notebook takes the raw data found in multiple files within the "raw" folder and places them all into one file for a given month. The data is separated into motor vehicle theft and theft and finally geocoded so that it can be used in the map construction.

In [7]:
import pandas as pd
import os
import geocoder
import requests

In [8]:
for folder in os.listdir('./raw'):
    file_list = []
    for file in os.listdir('./raw/' + folder):
        if(file.endswith('.xls')):
            data = pd.read_excel('./raw/' + folder + '/' + file)
            file_list.append(data)
   
    df = pd.concat(file_list, ignore_index=True)
    
    columns = ['Incident Number','Date', 'Time', 'Police District','Offense 1',
          'Offense 2', 'Offense 3', 'Offense 4', 'Offense 5', 'Location']
    df.columns = columns
    
    df = df[df['Offense 1'].notnull()]
    df = df[df['Location'].notnull()]
    
    df.to_csv('./raw/' + folder + 'Complete.csv', sep=';', index = False)

In [9]:
def address_to_latlong(address):
    '''Returns the latitude and longitude for a given address'''
    address += ", Milwaukee, WI"
    location = geocoder.arcgis(address, session = session)
    
    if not location.latlng:
        print("Couldn't Parse: "  + address )
    else:
        print(f'Parsed: {address} - {location.latlng}')
    return location.latlng

In [10]:
for file in os.listdir('./raw/'):
    if(file.endswith('.csv')):
        df = pd.read_csv('./raw/'+ file, sep =';')
    
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
              (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
              (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
              (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
              (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                df = df.drop(off_1[0])
 
        df['Address'] = df['Location'].copy()
            
        theft = df.copy()
        m_theft = df.copy()
        
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'motor vehicle theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
                (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
                (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
                (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
                (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                m_theft = m_theft.drop(off_1[0])
            else:
                theft = theft.drop(off_1[0])
                
        
        with requests.Session() as session:
            # This is likely to be very slow
            # depending on the geocoding service used
            theft['Location'] = theft['Location'].apply(address_to_latlong)
            m_theft['Location'] = m_theft['Location'].apply(address_to_latlong)
            
        theft.to_csv('./' + 'theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        m_theft.to_csv('./' + 'm_theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        print(f'Finished {file}')

Parsed: 6161 N 39TH ST, Milwaukee, WI - [43.13026858673322, -87.96091963921731]
Parsed: 2900 W CAMERON AV, Milwaukee, WI - [43.10728348600597, -87.94926947155058]
Parsed: 4129 N 12TH ST, Milwaukee, WI - [43.09197303136361, -87.92578963200395]
Parsed: 4210 W MONROVIA WA, Milwaukee, WI - [43.12839466011471, -87.96495144249708]
Parsed: 4202 W HAMPTON AV, Milwaukee, WI - [43.104683460470746, -87.96527999999999]
Parsed: 5070 N 35TH ST, Milwaukee, WI - [43.109657968636384, -87.95642589901388]
Parsed: 2524 W GLENDALE AV, Milwaukee, WI - [43.10087126054742, -87.94618513375163]
Parsed: 5028 N 47TH ST, Milwaukee, WI - [43.108883329447764, -87.97134341517136]
Parsed: 2522 W CAPITOL DR, Milwaukee, WI - [43.08985048600596, -87.94537164308379]
Parsed: 7835 N GREEN BAY RD, Milwaukee, WI - [43.160162376968444, -87.94714607438186]
Parsed: 3720 W VILLARD AV, Milwaukee, WI - [43.111940529863055, -87.95949372300768]
Parsed: 2701 W ROOSEVELT DR, Milwaukee, WI - [43.09510847013696, -87.94716283236194]
Parse

Parsed: 2626 N 55TH ST, Milwaukee, WI - [43.06674524562871, -87.9821233607827]
Parsed: 1527 N 49TH ST, Milwaukee, WI - [43.0508252514571, -87.97475960646872]
Parsed: 2151 N 57TH ST, Milwaukee, WI - [43.058304167638056, -87.9842206392173]
Parsed: 5801 W CENTER ST, Milwaukee, WI - [43.06800654674262, -87.98551127699231]
Parsed: 1509 N 47TH ST, Milwaukee, WI - [43.05043825145711, -87.97246010646872]
Parsed: 2735 N 54TH ST, Milwaukee, WI - [43.06880686372554, -87.9809975809335]
Parsed: 2763 N 54TH ST, Milwaukee, WI - [43.06927583819032, -87.98099063532217]
Parsed: 7261 W APPLETON AV, Milwaukee, WI - [43.08798261697164, -88.002679325033]
Parsed: 5920 W NORTH AV, Milwaukee, WI - [43.06073751875454, -87.98726266763806]
Parsed: 9000 W WISCONSIN AV, Milwaukee, WI - [43.03848938965115, -88.0248012987741]
Parsed: 8855 W WISCONSIN AV, Milwaukee, WI - [43.038382474680326, -88.02313707095895]
Parsed: 3902 N 75TH ST, Milwaukee, WI - [43.08821916180969, -88.00588591185313]
Parsed: 440 S 75TH ST, Milwa

Parsed: 1903 W WHITE ST, Milwaukee, WI - [43.012813996898025, -87.9375117464477]
Parsed: 723 W HISTORIC MITCHELL ST, Milwaukee, WI - [43.01225851399403, -87.920712]
Parsed: 1924 S 13TH ST, Milwaukee, WI - [43.00959507799067, -87.9281559190665]
Parsed: 2329 S 9TH PL, Milwaukee, WI - [43.00215294754457, -87.92364754818492]
Parsed: 200 E OREGON ST, Milwaukee, WI - [43.0284206544509, -87.90975217621998]
Parsed: 2487 S 6TH ST, Milwaukee, WI - [42.99957067055226, -87.91888955539827]
Parsed: 2111 S 14TH ST, Milwaukee, WI - [43.006167450458776, -87.92974848990112]
Parsed: 2674 S 13TH ST #10, Milwaukee, WI - [42.99597858090485, -87.92859542627986]
Parsed: 2210 S 7TH ST, Milwaukee, WI - [43.00424021426511, -87.92004352063047]
Parsed: 410 S 3RD ST, Milwaukee, WI - [43.02699288481736, -87.913974407958]
Parsed: 900 W HISTORIC MITCHELL ST, Milwaukee, WI - [43.012374148088995, -87.92278364808901]
Parsed: 4601 S 1ST ST #340, Milwaukee, WI - [42.960384000000005, -87.91112906376547]
Parsed: 4530 S 1ST S

Parsed: 1851 N 14TH ST, Milwaukee, WI - [43.05561277990651, -87.93015056650677]
Parsed: 2658 N 28TH ST, Milwaukee, WI - [43.06725866472388, -87.94860588963614]
Parsed: 2520 N 28TH ST, Milwaukee, WI - [43.06471141326679, -87.94863436078269]
Parsed: 2860 N 11TH ST, Milwaukee, WI - [43.0707774970858, -87.92529489353127]
Parsed: 2029 N 28TH ST, Milwaukee, WI - [43.05699941909515, -87.94875763200395]
Parsed: 2201 W NORTH AV, Milwaukee, WI - [43.06044934898673, -87.94010926031346]
Parsed: 1855 N 39TH ST, Milwaukee, WI - [43.054380754371294, -87.96242010978695]
Parsed: 1832 N 29TH ST, Milwaukee, WI - [43.0545777485429, -87.94988243738835]
Parsed: 4415 W GARFIELD AV, Milwaukee, WI - [43.059147226672835, -87.96908874969641]
Parsed: 2467 W GARFIELD AV, Milwaukee, WI - [43.05912750678069, -87.94445441909515]
Parsed: 2218 N 26TH ST, Milwaukee, WI - [43.059663, -87.94622588631792]
Parsed: 1800 W MEINECKE AV, Milwaukee, WI - [43.06226046768411, -87.93530958090484]
Parsed: 2524 W GARFIELD AV, Milwauk

Parsed: 2800 N 29TH ST, Milwaukee, WI - [43.069772217499, -87.94977303380104]
Parsed: 3800 W CENTER ST, Milwaukee, WI - [43.068012218731866, -87.96106971873186]
Parsed: 2316 W FOND DU LAC AV, Milwaukee, WI - [43.06276344071645, -87.94154632339314]
Parsed: 4329 W GARFIELD AV, Milwaukee, WI - [43.059201517889164, -87.96868183236194]
Parsed: 2107 N 36TH ST, Milwaukee, WI - [43.05771941909515, -87.9587900809335]
Parsed: 2043 N 24TH PL, Milwaukee, WI - [43.05727803136361, -87.94385756650678]
Parsed: 2400 W FOND DU LAC AV, Milwaukee, WI - [43.06353794683877, -87.94250168845294]
Parsed: 8665 W FOND DU LAC AV, Milwaukee, WI - [43.12330214764276, -88.01906286707242]
Parsed: 6509 W FLORIST AV, Milwaukee, WI - [43.12666854674262, -87.99258166763806]
Parsed: 5200 W FOND DU LAC AV, Milwaukee, WI - [43.09113226327063, -87.97766169400542]
Parsed: 6801 W VILLARD AV, Milwaukee, WI - [43.112085539529254, -87.99642058673322]
Parsed: 9331 W SILVER SPRING DR, Milwaukee, WI - [43.11940554674263, -88.0292608

...

Parsed: 312 N 41ST ST, Milwaukee, WI - [43.03404900000001, -87.96525340074464]
Parsed: 6019 W BLUE MOUND RD, Milwaukee, WI - [43.03640651009891, -87.98814838190317]
Parsed: 5100 W BURLEIGH ST, Milwaukee, WI - [43.07532150043268, -87.97774147446478]
Parsed: 5916 W SIEGFRIED PL, Milwaukee, WI - [43.02645546047075, -87.98698269317329]
Parsed: 5020 W VLIET ST, Milwaukee, WI - [43.049012019796095, -87.97647330317842]
Parsed: 5300 W CENTER ST, Milwaukee, WI - [43.068075218811956, -87.979905]
Parsed: 100 N MITCHELL BL, Milwaukee, WI - [43.028781976348085, -87.97877566764105]
Parsed: 1050 N 47TH ST, Milwaukee, WI - [43.04343574854292, -87.97256590074464]
Parsed: 1453 N 52ND ST, Milwaukee, WI - [43.050222251457086, -87.97881912479058]
Parsed: 5834 W LISBON AV, Milwaukee, WI - [43.067425598130185, -87.9860363480294]
Parsed: 5703 W HADLEY ST, Milwaukee, WI - [43.0697457788649, -87.98432858966021]
Parsed: 5427 W KEEFE AV, Milwaukee, WI - [43.08259952842077, -87.98181283236194]
Parsed: 3044 N 53RD 

Parsed: 123 W OKLAHOMA AV, Milwaukee, WI - [42.98748519044136, -87.91031934493097]
Parsed: 2746 S 10TH ST, Milwaukee, WI - [42.99448310352588, -87.92493443738836]
Parsed: 500 W LINCOLN AV, Milwaukee, WI - [43.00286751875456, -87.91710241326678]
Parsed: 2961 S 10TH ST, Milwaukee, WI - [42.99057958673322, -87.92509901543632]
Parsed: 2817 N 11TH LA, Milwaukee, WI - [43.07002241909515, -87.92603964253553]
Parsed: 2470 N 50TH ST, Milwaukee, WI - [43.06403700000001, -87.9761934190665]
Parsed: 3403 W LOCUST ST, Milwaukee, WI - [43.07147303715159, -87.95627555517007]
Parsed: 2477 W FOND DU LAC AV, Milwaukee, WI - [43.064951020077224, -87.94445214719498]
Parsed: 3732 W WISCONSIN AV #300, Milwaukee, WI - [43.03888818172833, -87.96084796095937]
Parsed: 1400 W CHERRY ST, Milwaukee, WI - [43.050048195901134, -87.93012319590115]
Parsed: 1447 N 38TH ST, Milwaukee, WI - [43.04980800000001, -87.9607981392173]
Parsed: 2236 N 15TH ST, Milwaukee, WI - [43.05967222009349, -87.93135288963613]
Parsed: 2225 N

Parsed: 3118 N HACKETT AV, Milwaukee, WI - [43.0750789103526, -87.87674487131426]
Parsed: 2535 N STOWELL AV, Milwaukee, WI - [43.06455125728547, -87.87942005539827]
Parsed: 1940 N PROSPECT AV #40, Milwaukee, WI - [43.05540330861574, -87.88831684315332]
Parsed: 3007 N MURRAY AV, Milwaukee, WI - [43.07306416763808, -87.88546857372015]
Parsed: 1320 N MARTIN L KING JR DR, Milwaukee, WI - [43.047064844984206, -87.91434854392124]
Parsed: 817 E PEARSON ST, Milwaukee, WI - [43.053704488458806, -87.90153333527613]
Parsed: 801 E KNAPP ST, Milwaukee, WI - [43.04685823905082, -87.9021528846646]
Parsed: 2311 E LINNWOOD AV, Milwaukee, WI - [43.07277749956732, -87.88118352844941]
Parsed: 1130 E KANE PL, Milwaukee, WI - [43.056180363556365, -87.897124959965]
Parsed: 1561 N VAN BUREN ST, Milwaukee, WI - [43.050095716569345, -87.90349918579305]
Parsed: 3129 N OAKLAND AV, Milwaukee, WI - [43.07544019900166, -87.88790060646872]
Parsed: 2457 N OAKLAND AV, Milwaukee, WI - [43.06269625145711, -87.88808960646

Parsed: 3214 N PALMER ST, Milwaukee, WI - [43.076762245628714, -87.90873240463978]
Parsed: 3513 N TEUTONIA AV, Milwaukee, WI - [43.082073702464115, -87.93475235417492]
Parsed: 1505 W COLUMBIA ST, Milwaukee, WI - [43.07410147403208, -87.93024252844941]
Parsed: 2825 N HOLTON ST, Milwaukee, WI - [43.07013033527613, -87.9053291253675]
Parsed: 3225 N 12TH ST, Milwaukee, WI - [43.07718495278099, -87.92625164498709]
Parsed: 801 W RING ST, Milwaukee, WI - [43.077540365751254, -87.92132181950446]
Parsed: 3118 N 11TH ST, Milwaukee, WI - [43.075431387731555, -87.92492288631792]
Parsed: 3171 N 12TH ST, Milwaukee, WI - [43.07642986372554, -87.9262505809335]
Parsed: 2430 W HOPKINS ST, Milwaukee, WI - [43.08017076132486, -87.9428335563219]
Parsed: 2210 N PALMER ST, Milwaukee, WI - [43.059401413266784, -87.90958292627985]
Parsed: 2430 W HOPKINS ST, Milwaukee, WI - [43.08017076132486, -87.9428335563219]
Parsed: 3345 N 6TH ST, Milwaukee, WI - [43.07984094754457, -87.91854056650678]
Parsed: 1930 W ATKINS

Parsed: 3200 W NATIONAL AV, Milwaukee, WI - [43.0218635115412, -87.95406186663973]
Parsed: 2623 W LINCOLN AV, Milwaukee, WI - [43.00314248845882, -87.94734333527613]
Parsed: 2113 W GREENFIELD AV, Milwaukee, WI - [43.016976546742626, -87.9400111902591]
Parsed: 1227 S 30TH ST, Milwaukee, WI - [43.018397664723864, -87.9515000809335]
Parsed: 5335 W SILVER SPRING DR, Milwaukee, WI - [43.11925049235396, -87.97957183236194]
Parsed: 6613 N 84TH ST, Milwaukee, WI - [43.13807879571822, -88.01639751918724]
Parsed: 8900 W MILL RD, Milwaukee, WI - [43.13432256720751, -88.02303197019448]
Parsed: 9015 N 96TH ST #10, Milwaukee, WI - [43.1827729930751, -88.03118309371543]
Parsed: 8947 N 96TH ST, Milwaukee, WI - [43.18134371218763, -88.03176066573347]
Parsed: 8701 W HELENA ST, Milwaukee, WI - [43.14604053814428, -88.01979358050082]
Parsed: 5854 N 62ND ST, Milwaukee, WI - [43.124255717179295, -87.98824485356933]
Parsed: 5410 W PRESIDIO LA #B, Milwaukee, WI - [43.14682531914589, -87.97974863811267]
Parsed